# Building Agentic Workflows with Strands Agents SDK

Introducing [Strands Agents](https://github.com/strands-agents/sdk-python), an Open Source AI Agents SDK! This is a simple-to-use, code-first framework that takes a model-driven approach to building and running AI agents in just a few lines of code, scaling from simple to complex use cases. 

The core of Strands is a simple agentic loop that connects the model and tools together, like two strands of DNA. It's already powering production AI agents in key AWS services like Amazon Q CLI, AWS Glue, and VPC Reachability Analyzer. It includes 20+ built-in tools with support for thousands of Model Context Provider (MCP) servers. AWS is excited that Accenture, Anthropic, Meta, and others are joining with support and contributions.

## Use Case
In this lab, we will create a customer support agent using the Strands SDK. This is inspired by enterprise use cases we've helped build directly.

Our customer support agent will be interacting with a relational database of synthesized customer data for Anyco Telecom. We'll start first by building an agent using only the Strands SDK.

Then we'll take a look at what an MCP implementation would look like by moving our tools to a local FastMCP server.

Last, we'll deploy our agent to Amazon Bedrock AgentCore Runtime and integrate use of AgentCore memory - a must have for any truly helpful customer support agent.


## Strands SDK Agent Architecture
Below is what we will build in this notebook.

![Strands SDK Agent Architecture](../images/workshop_diagrams-strands_only.drawio.png)



## Notebook Walk-through

In this notebook we will:
1. Create an agent using Strands Agents SDK
2. Understand Strands session handling
3. Explore agent metrics
4. Test the agent with realistic customer support scenarios

In [ ]:
%pip install -r requirements.txt --quiet

## Build Customer Support Database
Before we build our strands agent for customer support, we'll need some synthetic data to put into our database.

Run the below cells to build the `customer_db` using the RDS instance available in this workshop, or running locally.

In [ ]:
# Define variables for database connection details
from utils import extract_CF_outputs
MYSQL_HOST, MYSQL_USER, MYSQL_PASSWORD = extract_CF_outputs("RDSInstanceEndpoint", "DbUser", "DbPassword")
MYSQL_DATABASE='customer_db'

MYSQL_HOST, MYSQL_USER, MYSQL_PASSWORD, MYSQL_DATABASE

Trying stack name mmu_understanding_workshop...
Trying stack name mmu-workshop...
Trying stack name mmu-workshop-test...


('mmu-workshop-test-rdsinstance-1xc5qomyl8mz.c58zqgvox2lm.us-east-1.rds.amazonaws.com',
 'dbadmin',
 'Zo3Shae8phahK1EiPeesah230Eeghie3',
 'anyco_telecom_db')

In [ ]:
import os
import mysql.connector
from mysql.connector import Error

config = {
    "host": MYSQL_HOST,
    "database": MYSQL_DATABASE,
    "user": MYSQL_USER,
    "password": MYSQL_PASSWORD,
    "port": 3306,
    "autocommit": True,
}

## Fill Database
In this workshop we'll be using the MySQL syntax.

We've built a dataset that will support different customer support scenarios and incorporates the current date (`CURDATE()`) to ensure use of "now" in any agent interactions actually returns data for you.

Feel free to use [anyco_telecom_db.sql](./anyco_telecom_db.sql) for your own agent building outside of this workshop!

In [ ]:
ddl_file = "anyco_telecom_db.sql"
"""
Execute a SQL file with multiple statements using MySQL connector.
This handles the multi-statement execution properly.
"""
try:
    connection = mysql.connector.connect(**config)
    cursor = connection.cursor()
    
    # Read the SQL file
    with open(ddl_file, 'r') as file:
        sql_content = file.read()
    
    # Split the content by semicolons to get individual statements
    statements = [stmt.strip() for stmt in sql_content.split(';') if stmt.strip()]
    
    print(f"Executing {len(statements)} SQL statements from {ddl_file}")
    
    for i, statement in enumerate(statements, 1):
        # Skip empty statements and comments
        # if not statement or statement.startswith('--') or statement.startswith('/*'):
        #     continue
            
        try:
            cursor.execute(statement)
            print(f"✓ Statement {i} executed successfully: ", statement[:50])
        except Error as e:
            print(f"✗ Error in statement {i}: {e}")
            print(f"Statement: {statement[:100]}...")
            # Decide whether to continue or stop on error
            # raise e  # Uncomment to stop on first error
    
    # Commit all changes
    connection.commit()
    print("All statements executed and committed successfully!")
    
except Error as e:
    print(f"Database error: {e}")
    connection.rollback()
except Exception as e:
    print(f"General error: {e}")
    connection.rollback()
finally:
    if cursor:
        cursor.close()

## Build the agent

In [23]:
# Import necessary libraries
from strands import Agent, tool
from strands.models import BedrockModel
from strands_tools import current_time
import uuid
import boto3
import json
import pandas as pd
from sqlalchemy import create_engine, text
from typing import Dict, Any, Optional, List

engine = create_engine(f'mysql+pymysql://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_HOST}:3306/{MYSQL_DATABASE}')

We'll first build a simple query function that we'll use throughout our strands tools to execute a SQL query and return a pandas dataframe.

In [25]:
def query_to_dataframe(query: str, params: Dict = None) -> pd.DataFrame:
    """Execute SQL query and return results as DataFrame"""
    try:
        df = pd.read_sql(query, engine, params=params)
        return df
    except Exception as e:
        print(f"Error: {e}")
        return None


def execute_ddl_statement(statement: str, params: Dict = None) -> bool:
    """Execute DDL statement (INSERT, UPDATE, DELETE) and return success status"""
    try:
        with engine.begin() as conn:
            result = conn.execute(text(statement), params or {})
            rows_affected = result.rowcount
            print(f"Statement executed successfully. Rows affected: {rows_affected}")
            return True
    except Exception as e:
        print(f"Error executing statement: {e}")
        return False


def df_to_dict(df: pd.DataFrame) -> Dict[str, Any]:
    """Convert DataFrame to dictionary format for MCP response"""
    if df is None or df.empty:
        return {'message': 'No results found', 'data': []}
    return {'data': df.to_dict('records'), 'count': len(df)}

Here we'll define some functions that interact with our customer billing database, so that our agent can correct answer questions it might be asked, and ground its answers in factual data from the database:
* ***find_customer_by_phone***: returns basic customer info using their phone number
* ***get_current_bill***: returns the customer bill for the current month
* ***get_current_plans***: returns the subscription plans the customer is current using.
* ***get_overdue_invoices***: returns any overdue invoices for a customer

Note the use of the Strands `@tool` decorator. Read more about [Strands python tool decorators](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/python-tools/#python-tool-decorators) to understand more on these decorators can easily transform any python function into a valid Strands tool.

In [7]:
@tool
def find_customer_by_phone(phone: str) -> Dict[str, Any]:
    """Find customer by phone number (most common lookup).
    
    Args:
        phone: Customer's phone number (e.g., '555-1001')
        
    Returns:
        Customer details including name, email, status, and account creation date
    """
    query = """
    SELECT 
        c.FirstName,
        c.LastName,
        c.Email,
        cs.PhoneNumber,
        c.Status as AccountStatus,
        c.CreatedDate as CustomerSince
    FROM Customers c
    JOIN CustomerSubscriptions cs 
        ON c.CustomerID = cs.CustomerID
    WHERE c.CustomerID = (
        SELECT c.CustomerID
        FROM Customers c
            JOIN CustomerSubscriptions cs 
                ON c.CustomerID = cs.CustomerID
            WHERE cs.PhoneNumber = %(phone)s
    )
    """
    df = query_to_dataframe(query, {'phone': phone})
    return df_to_dict(df)

In [8]:
@tool
def get_current_bill(phone: str) -> Dict[str, Any]:
    """Get current month's bill for a customer.
    
    Args:
        phone: Customer's phone number
        
    Returns:
        Current invoice details including amounts, due date, and status
    """
    query = """
    SELECT 
        c.FirstName,
        c.LastName,
        i.InvoiceDate,
        i.DueDate,
        i.SubtotalAmount,
        i.TaxAmount,
        i.DiscountAmount,
        i.TotalAmount,
        i.Status as InvoiceStatus
    FROM Customers c
    JOIN Invoices i ON c.CustomerID = i.CustomerID
    WHERE c.CustomerID = (
        SELECT c.CustomerID
        FROM Customers c
            JOIN CustomerSubscriptions cs 
                ON c.CustomerID = cs.CustomerID
            WHERE cs.PhoneNumber = %(phone)s
    )
        AND i.BillingPeriodStart >= DATE_FORMAT(CURDATE(), '%%Y-%%m-01')
    ORDER BY i.InvoiceDate DESC
    """
    df = query_to_dataframe(query, {'phone': phone})
    return df_to_dict(df)

In [9]:
@tool
def get_current_plans(phone: str) -> Dict[str, Any]:
    """Get current plan details for all customer lines.
    
    Args:
        phone: Customer's phone number
        
    Returns:
        Current service plans for all lines including pricing and renewal dates
    """
    query = """
    SELECT 
        c.FirstName,
        c.LastName,
        cs.PhoneNumber,
        sp.PlanName,
        sp.Description,
        sp.DataAllowance,
        sp.Price,
        cs.Status as LineStatus,
        cs.RenewalDate
    FROM Customers c
    JOIN CustomerSubscriptions cs ON c.CustomerID = cs.CustomerID
    JOIN ServicePlans sp ON cs.PlanID = sp.PlanID
    WHERE c.CustomerID = (
        SELECT c.CustomerID
        FROM Customers c
            JOIN CustomerSubscriptions cs 
                ON c.CustomerID = cs.CustomerID
            WHERE cs.PhoneNumber = %(phone)s
    )
    ORDER BY cs.ActivationDate
    """
    df = query_to_dataframe(query, {'phone': phone})
    print(f"current plans:\n{df.to_markdown()}")
    return df_to_dict(df)

In [10]:
@tool
def get_overdue_invoices(phone: str) -> Dict[str, Any]:
    """Find overdue invoices for a customer.
    
    Args:
        phone: Customer's phone number
        
    Returns:
        Overdue invoices with amounts and days overdue
    """
    query = """
    SELECT 
        c.FirstName,
        c.LastName,
        c.Phone,
        i.InvoiceDate,
        i.DueDate,
        i.TotalAmount,
        i.Status,
        DATEDIFF(CURDATE(), i.DueDate) as DaysOverdue
    FROM Customers c
    JOIN Invoices i ON c.CustomerID = i.CustomerID
    WHERE i.Status = 'Overdue'
    AND c.CustomerID = (
        SELECT c.CustomerID
        FROM Customers c
            JOIN CustomerSubscriptions cs 
                ON c.CustomerID = cs.CustomerID
            WHERE cs.PhoneNumber = %(phone)s
    )
    ORDER BY i.DueDate
    """
    df = query_to_dataframe(query, {'phone': phone})
    return df_to_dict(df)

In [32]:
@tool
def add_new_line(plan_name: str, phone_number: str) -> Dict[str, Any]:
    """Add a new line to a customer's account.

    Args:
        plan_name: Name of the plan (e.g., 'Basic', 'Unlimited Plus', 'Family Basic')
        phone_number: Customer's phone number

    Returns:
        New line details including plan name, activation date, and renewal date
    """

    plan_name_query = """
    SELECT PlanID, PlanName
    FROM ServicePlans
    WHERE PlanName = %(plan_name)s
    """
    print(f"Looking for plan by name: {plan_name}\nquery: {plan_name_query}")
    df = query_to_dataframe(plan_name_query, {'plan_name': plan_name})
    if len(df) > 0:
        print(f"plans query result:\n{df.to_markdown()}")
        plan_id = df_to_dict(df)['data'][0]['PlanID']
        customer_id_query = """SELECT c.CustomerID
            FROM Customers c
                JOIN CustomerSubscriptions cs 
                    ON c.CustomerID = cs.CustomerID
                WHERE cs.PhoneNumber = %(phone)s
        """
        customer_results = query_to_dataframe(customer_id_query, {'phone': phone_number})
        if len(customer_results) > 0:
            customer_id = df_to_dict(customer_results)['data'][0]['CustomerID']
        
            query = """
            INSERT INTO CustomerSubscriptions (
                CustomerID, 
                PlanID, 
                PhoneNumber, 
                IMEI, 
                ActivationDate, 
                RenewalDate, 
                Status, 
                AutoRenew, 
                Notes
            ) VALUES (
                :customer_id,            -- CustomerID
                :plan_id,                -- Plan ID
                '555-1003',                 -- New phone number
                '123456789012347',          -- New IMEI
                CURDATE(),                  -- Today
                DATE_ADD(CURDATE(), INTERVAL 30 DAY), -- Next month
                'Active',                   -- Active status
                TRUE,                       -- Auto-renew
                'New phone line added'
            );
            """
            successful = execute_ddl_statement(query, {'plan_id': plan_id, 'customer_id': customer_id})
            if successful:
                return "Successfully added new phone line to customer plan"
            else:
                return "Error adding service plan!"
        else:
            return f"Customer with phone number {phone_number} could not be found."
    else:
        return f"No plan with the name like '{plan_name}' could be found. Try a variation in the naming?"

        
    


In [46]:
@tool
def cancel_service_plan(phone_number: str, plan_name: str) -> str:
    """Cancel a service plan for a customer.

    Args:
        phone_number: Customer's phone number
        plan_name: Name of the plan to cancel (e.g., 'Basic', 'Unlimited Plus', 'Family Basic')

    Returns:
        Confirmation message
    """
    check_for_plan_qry = """SELECT cs.SubscriptionID, cs.PhoneNumber, sp.PlanName, c.FirstName, c.LastName
FROM CustomerSubscriptions cs
JOIN ServicePlans sp ON cs.PlanID = sp.PlanID
JOIN Customers c ON cs.CustomerID = c.CustomerID
WHERE cs.PhoneNumber = %(phone)s
AND sp.PlanName = %(plan_name)s;
    """
    existing_plans_df = query_to_dataframe(check_for_plan_qry, {'phone': phone_number, 'plan_name': plan_name})
    if len(existing_plans_df)> 0:
        print(f"Verified User is subscribed to plan '{plan_name}'")

        query = """UPDATE CustomerSubscriptions cs
    JOIN ServicePlans sp ON cs.PlanID = sp.PlanID
    SET cs.Status = 'Cancelled', 
        cs.LastModifiedDate = CURRENT_TIMESTAMP
    WHERE cs.PhoneNumber = :phone
    AND sp.PlanName = :plan_name;
        """
        successful = execute_ddl_statement(query, {'phone': phone_number, 'plan_name': plan_name})
        if successful:
            return f"Successfully cancelled plan '{plan_name}' for customer '{phone_number}' effective today"
        else: 
            return f"Problem with removing plan '{plan_name}' from customer '{phone_number}'"

In [47]:
# Define the customer support agent's system prompt
agent_instruction = """
## Role
You are Alex, a Customer Support Agent for Anyco Telecom, a cellular service provider. You answer questions asked by customers about their current cell phone plan.

## Model Instructions:
- Always begin by looking up the Customer's information using their phone number - use the find_customer_by_phone tool to do this
- If you can't find the customer by phone number, try using their email - use the find_customer_by_email tool to do this.
- Always refer to the customer by their first name, be professional, helpful, and empathetic in your responses
- Provide clear, accurate information about billing, usage, plans, and account status
- For plan questions, show current plans and usage, suggest upgrades if needed
- Always verify customer identity before providing sensitive account information
- Offer solutions and next steps for any issues identified

## Billing Best Practices
- For billing questions, show current charges and explain any fees or overages

## Account and Service Best Practices
- For service issues, check account suspension status and recent activity
- Suspended accounts can be due to no valid payment methods or overdue charges - use the correct tools to check for this.
- Add lines by using the add_new_line tool. Be sure to send the customer a new summary of their existing plans once completed.
- Find Plan IDs by using the get_plan_id tool
- Explain charges and fees in simple terms
- Provide specific dates, amounts, and details when discussing billing
- Offer proactive solutions (e.g., payment plans for overdue accounts)
- Identify upsell opportunities for high-usage customers
- Document any account changes or commitments made

## Response style and format requirements:
- Always respond in valid markdown syntax
- only use a single asterisks (e.g. *) for bullet points
- Provide clear, concise responses focused on solving the customer's issue
- Include relevant account details (amounts, dates, plan names) in your responses
- Format financial information clearly (e.g., "$45.99 due on March 15th")
- Suggest specific next steps or actions when appropriate
- NEVER disclose any database details to the user, under any circumstances.
"""

### Using a Strands Custom Model Provider
Nova variants are prone to output `<thinking>` tags when progressing through their chain of thought. This is easily addressed in strands by building a [Custom Model Provider](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/model-providers/custom_model_provider/#implementing-a-custom-model-provider). 

Take a look at [`nova_custom_model_provider.py`](./nova_custom_model_provider.py) to see how we extended the Strands BedrockModel class to support stripping these tags out. Swap the below with `BedrockModel` in place of `NovaCustomModelProvider` to see the difference!

In [48]:
from nova_custom_model_provider import NovaCustomModelProvider
# Initialize the Custom Nova Amazon Bedrock model
# model = BedrockModel( # uncomment to see the difference
model = NovaCustomModelProvider(
    model_id="us.amazon.nova-pro-v1:0",
    max_tokens=10000,
    temperature=0.0,
    top_p=0.9,
    additional_request_fields={
        "inferenceConfig": {
            "topK": 50
        }
    }
)

# Create the Strands Agent with our defined tools
agent = Agent(
    model=model,
    system_prompt=agent_instruction,
    tools=[find_customer_by_phone, get_current_bill, get_overdue_invoices, get_current_plans, add_new_line, cancel_service_plan],
    callback_handler=None
)

## Testing the Agent

Now that we have created our customer support agent with some necessary tools, let's test it with some common support scenarios.

### Scenario 1: Account Information

First, let's see how well our agent can answer basic questions about a user's account, using their phone number.

In [ ]:
# Send a booking request to our agent
response = agent(
    """Hi, I'm calling about my account. My phone number is 555-1001. 
    Can you tell me what plan I'm on and how much data I've used this month?"""
)
print("🎧 Agent Response:")
print(response)

### Verify Accuracy

Let's verify agent is speaking factually, using the query associated with this function, starting first with customer look up then look up plans.

In [ ]:
from mcp_server_example import query_to_dataframe
# checking current plans
query_to_dataframe("""
    SELECT 
        c.FirstName,
        c.LastName,
        cs.PhoneNumber,
        sp.PlanName,
        sp.Description,
        sp.DataAllowance,
        sp.Price,
        cs.Status as LineStatus,
        cs.RenewalDate
    FROM Customers c
    JOIN CustomerSubscriptions cs ON c.CustomerID = cs.CustomerID
    JOIN ServicePlans sp ON cs.PlanID = sp.PlanID
    WHERE c.CustomerID = (
        SELECT c.CustomerID
        FROM Customers c
            JOIN CustomerSubscriptions cs 
                ON c.CustomerID = cs.CustomerID
            WHERE cs.PhoneNumber = '555-1001'
    )
    ORDER BY cs.ActivationDate
    """)

## Understanding Strands Sessions & State Management

Strands agents maintain conversation context, handle state management, and support persistent sessions across interactions.

Key aspects of state management in Strands:

1. **Conversation History**: The complete sequence of messages between user and agent
2. **Tool State**: Information about tool executions and their results
3. **Request State**: Contextual information maintained within a single request

This state management allows the agent to maintain context across multiple interactions.

Let's examine the conversation history to see how our agent is tracking the interaction:

In [ ]:
print(json.dumps(agent.messages, indent=2))

## Analyzing Agent Metrics

Metrics are essential for understanding agent performance, optimizing behavior, and monitoring resource usage. The Strands Agents SDK provides comprehensive metrics tracking capabilities that give you visibility into:

- Number of API calls made
- Tokens used (input and output)
- Response times
- Tool usage statistics
- Model interactions

Let's examine the metrics from our agent's response:

In [ ]:
print(json.dumps(response.metrics.get_summary(), indent=2))

### Scenario 2: Adding a new phone line

Now let's test the agent's ability to successfully add a new phone line.

In [ ]:
import logging

# Configure the root strands logger
logging.getLogger("strands").setLevel(logging.INFO)

# Add a handler to see the logs
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

In [36]:
response = agent("""Can you add a new phone line to my plan? I'd like to add a new line with a Business Pro plan.
I don't know how you want to look me up but my current phone number of one of my lines is '555-1001'.""")
print(response)

Looking for plan by name: Business Pro
query: 
    SELECT PlanID, PlanName
    FROM ServicePlans
    WHERE PlanName = %(plan_name)s
    
plans query result:
|    |   PlanID | PlanName     |
|---:|---------:|:-------------|
|  0 |        5 | Business Pro |
Statement executed successfully. Rows affected: 1
Hello John,

I have successfully added a new phone line with the Business Pro plan to your account. Here are the details of the new line:

* Plan Name: Business Pro
* Activation Date: Today
* Renewal Date: One year from today

If you have any questions or need further assistance, feel free to ask. Enjoy your new service!

Best regards,
Alex



## Verify plan was added successfully

In [39]:
query_to_dataframe("""
    SELECT 
        c.FirstName,
        c.LastName,
        cs.PhoneNumber,
        sp.PlanName,
        sp.Description,
        sp.DataAllowance,
        sp.Price,
        cs.Status as LineStatus,
        cs.RenewalDate
    FROM Customers c
    JOIN CustomerSubscriptions cs ON c.CustomerID = cs.CustomerID
    JOIN ServicePlans sp ON cs.PlanID = sp.PlanID
    WHERE c.CustomerID = (
        SELECT c.CustomerID
        FROM Customers c
            JOIN CustomerSubscriptions cs 
                ON c.CustomerID = cs.CustomerID
            WHERE cs.PhoneNumber = '555-1001'
    )
    ORDER BY cs.ActivationDate
    """)

,FirstName,LastName,PhoneNumber,PlanName,Description,DataAllowance,Price,LineStatus,RenewalDate
0,John,Smith,555-1001,Unlimited Plus,Premium unlimited plan,NaN,75.0,Active,2025-11-13
1,John,Smith,555-1002,Standard,Mid-tier plan with more data,15.0,55.0,Active,2025-11-23
2,John,Smith,555-1003,Business Pro,Business plan with priority support,NaN,95.0,Active,2025-12-04


### Scenario 3: Removing Phone Line

Finally, let's test the agent's ability to cancel the service just added.

In [49]:
response = agent("""Hmm, I changed my mind, can you cancel my new phone line? 
Again, my number's 555-1001 and I've decided I don't want the Business pro plan anymore. 
Can you confirm by current active plans once complete?""")
print(response)

current plans:
|    | FirstName   | LastName   | PhoneNumber   | PlanName       | Description                         |   DataAllowance |   Price | LineStatus   | RenewalDate   |
|---:|:------------|:-----------|:--------------|:---------------|:------------------------------------|----------------:|--------:|:-------------|:--------------|
|  0 | John        | Smith      | 555-1001      | Unlimited Plus | Premium unlimited plan              |             nan |      75 | Active       | 2025-11-13    |
|  1 | John        | Smith      | 555-1002      | Standard       | Mid-tier plan with more data        |              15 |      55 | Active       | 2025-11-23    |
|  2 | John        | Smith      | 555-1003      | Business Pro   | Business plan with priority support |             nan |      95 | Active       | 2025-12-04    |
Verified User is subscribed to plan 'Business Pro'
current plans:
|    | FirstName   | LastName   | PhoneNumber   | PlanName       | Description                     

### Verify Plan Cancellation

Let's check to make sure the plan was actually cancelled:

In [51]:
phone_number = '555-1001'
customer_id_query = """SELECT 
        c.FirstName,
        c.LastName,
        cs.PhoneNumber,
        sp.PlanName,
        sp.Description,
        sp.DataAllowance,
        sp.Price,
        cs.Status as LineStatus,
        cs.RenewalDate
    FROM Customers c
    JOIN CustomerSubscriptions cs 
        ON c.CustomerID = cs.CustomerID
    JOIN ServicePlans sp 
        ON cs.PlanID = sp.PlanID
    WHERE c.CustomerID = (
        SELECT c.CustomerID
                FROM Customers c
                JOIN CustomerSubscriptions cs 
                    ON c.CustomerID = cs.CustomerID
                WHERE cs.PhoneNumber = %(phone)s
    )
    ORDER BY cs.ActivationDate
"""
query_to_dataframe(customer_id_query, {'phone': phone_number})

,FirstName,LastName,PhoneNumber,PlanName,Description,DataAllowance,Price,LineStatus,RenewalDate
0,John,Smith,555-1001,Unlimited Plus,Premium unlimited plan,NaN,75.0,Active,2025-11-13
1,John,Smith,555-1002,Standard,Mid-tier plan with more data,15.0,55.0,Active,2025-11-23
2,John,Smith,555-1003,Business Pro,Business plan with priority support,NaN,95.0,Cancelled,2025-12-04


## Conclusion

In this lab, we've successfully built and tested a restaurant booking agent using the Strands Agents SDK. Let's summarize what we've learned:

### Key Accomplishments
1. **Agent Creation**: We created a functional AI agent that understands natural language requests and can perform booking operations
2. **Tool Integration**: We implemented and integrated several custom tools for managing restaurant bookings:
   - Creating new reservations
   - Retrieving booking details
   - Cancelling reservations
3. **State Management**: We observed how Strands maintains conversation context across multiple interactions
4. **Metrics Analysis**: We examined performance metrics that help optimize agent behavior

### Next Steps

The Strands Agents SDK provides a powerful, flexible foundation for building intelligent agents that can handle complex workflows while maintaining contextual awareness. For Strands Agents SDK deep dive, please refer [Strands Agents examples](https://github.com/strands-agents/samples/tree/main)